In [136]:
import torch
import pandas as pd
import numpy as np

In [137]:
# applies the update given a set of parameters and a learning 
# rate lr. essentially a class used advance the parameters in the correct direction
 
class SGD():
    """Minibatch stochastic gradient descent"""

    def __init__(self, params, lr):
        self.params = params
        self.lr = lr

    def step(self):
        for param in self.params:
            print(f"Gradient: {param.grad}")
            param -= self.lr * param.grad

    def zero_grad(self):
        for param in self.params:
            if param.grad is not None:
                param.grad.zero_()

In [138]:
class HouseSale:

    def __init__(self, num_features, lr):
        self.w = torch.normal(100, 0.01, (num_features, 1), requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)
        self.lr = lr

    def forward(self, X):
        return torch.matmul(X, self.w) + self.b
    
    def loss(self, y_hat, y):
        return ((y_hat - y) ** 2 / 2).mean()

    def configure_optimizers(self):
        return SGD([self.w, self.b], self.lr)
    
    def training_step(self, batch):
        return self.loss(self.forward(batch[0]), batch[1])
    
    def check(self):
        print("X train")
        # print(self.x_train)
        print(self.x_train.shape)
        print("Y train")
        # print(self.y_train)
        print(self.y_train.shape)
        print("X val")
        # print(self.x_val)
        print(self.x_val.shape)
        print("Y val")
        # print(self.y_val)
        print(self.y_val.shape)


In [139]:
class Data:

    def __init__(self, batch_size):
        data = pd.read_csv('housing.csv')
        x = data.loc[:,["area"]]
        x = x[:].values
        y = data.loc[:,"price"]
        y = y[:].values
        y = y / y.sum()
        x = x / x.sum()
        self.x_train = torch.tensor(x[0:400], dtype=torch.float32, requires_grad=True)
        self.x_val = torch.tensor(x[400:500], dtype=torch.float32, requires_grad=True)
        self.y_train = torch.tensor(y[0:400], dtype=torch.float32, requires_grad=True)
        self.y_val = torch.tensor(y[400:500], dtype=torch.float32, requires_grad=True)
        self.batch_size = batch_size

    def train_dataloader(self):
        iters = self.x_train.shape[0] // self.batch_size
        for i in range(iters):
            yield [self.x_train[i:i+self.batch_size, :], self.y_train[i:i+self.batch_size].reshape((self.batch_size, 1))]

    def val_dataloader(self):
        iters = self.x_val.shape[0] // self.batch_size
        for i in range(0, iters):
            yield [self.x_val[i:i+self.batch_size, :], self.y_val[i:i+ self.batch_size].reshape((self.batch_size, 1))]
            

In [140]:
model = HouseSale(1, 0.5)
data = Data(20)
epochs = 5
gradient_clip_val = 0
sgd = model.configure_optimizers()

for i in range(epochs):
    for batch in data.train_dataloader():
        loss = model.training_step(batch)
        print(f"Loss: {loss}")
        sgd.zero_grad()
        with torch.no_grad():
            loss.backward()
            sgd.step()
            print(f"Weight: {model.w}")
            print(f"Bias: {model.b}")
print(f"Trained Weight: {model.w}")
print(f"Trained Bias: {model.b}")

Loss: 0.04260017350316048
Gradient: tensor([[0.0009]])
Gradient: tensor([0.2749])
Weight: tensor([[100.0044]], requires_grad=True)
Bias: tensor([-0.1375], requires_grad=True)
Loss: 0.013884807005524635
Gradient: tensor([[0.0005]])
Gradient: tensor([0.1320])
Weight: tensor([[100.0042]], requires_grad=True)
Bias: tensor([-0.2035], requires_grad=True)
Loss: 0.0070966435596346855
Gradient: tensor([[0.0003]])
Gradient: tensor([0.0629])
Weight: tensor([[100.0040]], requires_grad=True)
Bias: tensor([-0.2349], requires_grad=True)
Loss: 0.005341439973562956
Gradient: tensor([[0.0002]])
Gradient: tensor([0.0281])
Weight: tensor([[100.0040]], requires_grad=True)
Bias: tensor([-0.2490], requires_grad=True)
Loss: 0.005242545623332262
Gradient: tensor([[0.0001]])
Gradient: tensor([0.0089])
Weight: tensor([[100.0039]], requires_grad=True)
Bias: tensor([-0.2534], requires_grad=True)
Loss: 0.005292498040944338
Gradient: tensor([[0.0001]])
Gradient: tensor([0.0070])
Weight: tensor([[100.0038]], requires